pip install prophet --quiet - встановіть через термінал.
pip install pandas
pip install matplotlib

In [38]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt


Loading of DataSet, Some checks and data prep

In [39]:
# Load dataset
df = pd.read_csv('sales_dataset.csv')


In [40]:
# Show first few rows
df.head()

,sales_date,sku_id,dc_id,sales_count,weight,promo,product_name,category,location,is_weekend,month_name
0,2025-05-01,SKU005,1,7,0.56,True,Chicken,Meat,Kyiv,False,May
1,2025-05-02,SKU005,1,23,3.00,True,Chicken,Meat,Kyiv,False,May
2,2025-05-03,SKU004,1,2,1.68,True,Carrot,Vegetables,Kyiv,True,May
3,2025-05-04,SKU003,1,43,1.40,True,Bread,Bakery,Kyiv,True,May
4,2025-05-05,SKU003,3,26,4.87,True,Bread,Bakery,Odesa,False,May


In [41]:
# Check the column names in your dataset
df.columns

Index(['sales_date', 'sku_id', 'dc_id', 'sales_count', 'weight', 'promo',
       'product_name', 'category', 'location', 'is_weekend', 'month_name'],
      dtype='object')

In [42]:
#  Convert 'period' to datetime
df['sales_date'] = pd.to_datetime(df['sales_date'])

In [43]:
df = df[['sales_date', 'sku_id', 'dc_id', 'sales_count', 'weight', 'promo',
         'product_name', 'category', 'location', 'is_weekend', 'month_name']]

In [44]:
# Prepare results container
forecast_all = []

In [45]:
# Loop through each unique SKU
sku_list = df['sku_id'].unique()

for sku in sku_list:
    sku_df = df[df['sku_id'] == sku].copy()

    # Skip SKUs with too little data
    if len(sku_df) < 10:
        continue

    # Rename columns to match Prophet requirements
    sku_df.rename(columns={'sales_date': 'ds', 'sales_count': 'y'}, inplace=True)  # Заміна 'qty_total' на 'sales_count'

    # Drop rows with missing values in regressors
    sku_df = sku_df.dropna(subset=['promo', 'is_weekend', 'month_name'])  # Заміна на відповідні стовпці

    # Convert month_name to numeric month (e.g., 'May' -> 5)
    month_map = {
        "Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, "May": 5, "Jun": 6,
        "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12
    }
    sku_df['month_num'] = sku_df['month_name'].map(month_map)

    # Skip if less than 2 rows after cleaning
    if len(sku_df) < 2:
        continue

    # Initialize Prophet with additional regressors
    model = Prophet()
    model.add_regressor('promo')
    model.add_regressor('is_weekend')
    model.add_regressor('month_num')  # Використовуємо month_num замість month_name

    model.fit(sku_df)

    # Create future dataframe for 28 days
    future = model.make_future_dataframe(periods=28)

    # Add future regressor values (using last known value)
    future['promo'] = sku_df['promo'].iloc[-1]
    future['is_weekend'] = sku_df['is_weekend'].iloc[-1]
    future['month_num'] = sku_df['month_num'].iloc[-1]  # repeat last known value for future

    # Forecast
    forecast = model.predict(future)

    result = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(28).copy()
    result.rename(columns={
        'ds': 'date',
        'yhat': 'forecast_qty_total',
        'yhat_lower': 'forecast_qty_total_lower',
        'yhat_upper': 'forecast_qty_total_upper'
    }, inplace=True)

    result['sku_id'] = sku

    forecast_all.append(result)

16:41:25 - cmdstanpy - INFO - Chain [1] start processing
16:41:25 - cmdstanpy - INFO - Chain [1] done processing
16:41:25 - cmdstanpy - INFO - Chain [1] start processing
16:41:25 - cmdstanpy - INFO - Chain [1] done processing
16:41:25 - cmdstanpy - INFO - Chain [1] start processing
16:41:25 - cmdstanpy - INFO - Chain [1] done processing
16:41:25 - cmdstanpy - INFO - Chain [1] start processing
16:41:25 - cmdstanpy - INFO - Chain [1] done processing
16:41:26 - cmdstanpy - INFO - Chain [1] start processing
16:41:26 - cmdstanpy - INFO - Chain [1] done processing


Getting a result, saving to Csv.

In [46]:
#  Combine all forecasts
final_forecast_df = pd.concat(forecast_all, ignore_index=True)


In [47]:

# Save to CSV
final_forecast_df.to_csv('forecast_28_days_all_skus_with_regressors.csv', index=False)

In [48]:

# Show sample of result
final_forecast_df.head()

,date,forecast_qty_total,forecast_qty_total_lower,forecast_qty_total_upper,sku_id
0,2025-05-29,36.655778,23.129004,51.588920,SKU005
1,2025-05-30,59.649109,46.579712,74.816394,SKU005
2,2025-05-31,45.228966,30.472095,58.501733,SKU005
3,2025-06-01,46.538140,32.001691,59.672678,SKU005
4,2025-06-02,52.547277,38.073737,67.342904,SKU005
